In [41]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [42]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,0,Palembang,ID,2020-02-24 02:57:22,-2.92,104.75,85.21,69,3,6.73,light rain,0.50,0.0
1,1,Hurricane,US,2020-02-24 02:57:23,37.18,-113.29,51.80,75,0,6.93,clear sky,0.00,0.0
2,2,Pisco,PE,2020-02-24 02:57:23,-13.70,-76.22,77.00,78,75,8.05,broken clouds,0.00,0.0
3,3,Faanui,PF,2020-02-24 02:57:23,-16.48,-151.75,83.39,70,6,9.04,light rain,0.13,0.0
4,4,College,US,2020-02-24 02:57:24,64.86,-147.80,-2.00,70,20,0.87,few clouds,0.00,0.0


In [43]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [44]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = input("Do you want it to be raining? (yes/no) ")
snowing = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [45]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

if raining == "yes" and snowing == "yes":
    preferred_cities_df = preferred_cities_df.loc[(city_data_df["Snow Inches (last 3 hrs)"] > 0) | \
                                                  (city_data_df["Rain Inches (last 3 hrs)"] > 0)]
elif raining == "yes" and snowing == "no":
    preferred_cities_df = preferred_cities_df.loc[(city_data_df["Rain Inches (last 3 hrs)"] > 0) & \
                                                  (city_data_df["Snow Inches (last 3 hrs)"] == 0)]
elif raining == "no" and snowing == "yes":
    preferred_cities_df = preferred_cities_df.loc[(city_data_df["Rain Inches (last 3 hrs)"] == 0) & \
                                                  (city_data_df["Snow Inches (last 3 hrs)"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(city_data_df["Rain Inches (last 3 hrs)"] == 0) & \
                                                  (city_data_df["Snow Inches (last 3 hrs)"] == 0)]
    
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
2,2,Pisco,PE,2020-02-24 02:57:23,-13.70,-76.22,77.00,78,75,8.05,broken clouds,0.0,0.0
5,5,Busselton,AU,2020-02-24 02:57:24,-33.65,115.33,82.99,32,100,15.21,overcast clouds,0.0,0.0
7,7,Georgetown,MY,2020-02-24 02:57:25,5.41,100.34,86.00,58,20,4.70,few clouds,0.0,0.0
9,9,Mahebourg,MU,2020-02-24 02:57:26,-20.41,57.70,75.99,94,20,3.36,few clouds,0.0,0.0
17,17,San Andres,CO,2020-02-24 02:56:41,12.58,-81.70,77.00,88,75,11.41,broken clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,524,Kudat,MY,2020-02-24 03:02:08,6.88,116.85,82.40,78,75,5.82,broken clouds,0.0,0.0
534,534,Mayumba,GA,2020-02-24 03:02:13,-3.43,10.66,81.55,77,100,4.76,overcast clouds,0.0,0.0
551,551,Puerto Escondido,MX,2020-02-24 03:02:22,15.85,-97.07,80.60,88,40,8.05,scattered clouds,0.0,0.0
555,555,Malindi,KE,2020-02-24 03:02:23,-3.22,40.12,80.60,88,40,2.24,scattered clouds,0.0,0.0


In [46]:
preferred_cities_df.count()

City_ID                     130
City                        130
Country                     129
Date                        130
Lat                         130
Lng                         130
Max Temp                    130
Humidity                    130
Cloudiness                  130
Wind Speed                  130
Current Description         130
Rain Inches (last 3 hrs)    130
Snow Inches (last 3 hrs)    130
dtype: int64

In [47]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[['City', 'Country', "Current Description",'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
2,Pisco,PE,broken clouds,77.00,-13.70,-76.22,
5,Busselton,AU,overcast clouds,82.99,-33.65,115.33,
7,Georgetown,MY,few clouds,86.00,5.41,100.34,
9,Mahebourg,MU,few clouds,75.99,-20.41,57.70,
17,San Andres,CO,broken clouds,77.00,12.58,-81.70,


In [48]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [49]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [50]:
hotel_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
2,Pisco,PE,broken clouds,77.00,-13.70,-76.22,Hostal San Isidro Oficial
5,Busselton,AU,overcast clouds,82.99,-33.65,115.33,Observatory Guest House
7,Georgetown,MY,few clouds,86.00,5.41,100.34,Cititel Penang
9,Mahebourg,MU,few clouds,75.99,-20.41,57.70,Shandrani Beachcomber Resort & Spa
17,San Andres,CO,broken clouds,77.00,12.58,-81.70,Hotel Arena Blanca


In [51]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [54]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [55]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))